In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

# Overview of the Data


In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Selecting Albums to Purchase


In [3]:
%%sql
WITH track_genres AS
    (
     SELECT
        t.track_id,
        g.name genre_name
     FROM track t
     INNER JOIN genre g ON g.genre_id == t.genre_id
    ),
    us_customers AS
    (
     SELECT
        il.invoice_id,
        il.track_id
     FROM invoice i
     INNER JOIN invoice_line il ON i.invoice_id == il.invoice_id
     WHERE i.billing_country == 'USA'
    )

SELECT
    t.genre_name genre,
    COUNT(*) tracks_sold,
    ROUND(CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM us_customers) * 100, 2) percentage_sold
FROM us_customers uc
LEFT JOIN track_genres t ON t.track_id == uc.track_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

Done.


genre,tracks_sold,percentage_sold
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


The most sold genre in the USA are ROCK which is sold almost 4 times more as Punk. The second most sold is Punk at 12 percent then Metal at 11.8 percent and then Soul at 5.04 percent.
The record store should add Red Tone, Slim Jim Bites and Meteor and the Girls as these falls in the top sold category. Regal falls into Hip hop category which

# Analyzing Employee Sales Performance


In [4]:
%%sql
WITH purchases AS
    (
     SELECT
        c.customer_id,
        c.support_rep_id,
        SUM(i.total) total
     FROM customer c
     INNER JOIN invoice i ON i.customer_id == c.customer_id
     GROUP BY 1
    )
    
SELECT
    SUM(p.total) sales,
    e.first_name || ' ' || e.last_name name,
    e.hire_date 'Date of Joining'
FROM employee e
INNER JOIN purchases p ON e.employee_id == p.support_rep_id
GROUP BY 2
ORDER BY 1 DESC
LIMIT 10

Done.


sales,name,Date of Joining
1731.5099999999998,Jane Peacock,2017-04-01 00:00:00
1584.0000000000002,Margaret Park,2017-05-03 00:00:00
1393.92,Steve Johnson,2017-10-17 00:00:00


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds to the differences in their hiring dates.

# Analyzing Sales by Country

In [5]:
%%sql
WITH countries AS
    (
     SELECT
        c.country country,
        COUNT(DISTINCT(c.customer_id)) number_of_customers,
        SUM(i.total) total_sales,
        SUM(i.total) / COUNT(DISTINCT(c.customer_id)) AS avg_sales,
        AVG(i.total) avg_order_value
     FROM customer c
     INNER JOIN invoice i ON c.customer_id == i.customer_id
     GROUP BY 1
     ORDER BY 3 DESC
    ),
    group_countries AS
    (
     SELECT
        'other' country,
        COUNT(number_of_customers) number_of_customers,
        SUM(total_sales) total_sales,
        SUM(total_sales) / COUNT(number_of_customers)  avg_sales,
        AVG(avg_order_value) avg_order_value
     FROM countries
     WHERE number_of_customers == 1
    ),
    individual_countries AS 
    (
     SELECT * FROM countries
     WHERE number_of_customers > 1
    ),
    merged_data AS
    (
     SELECT * FROM individual_countries
     UNION
     SELECT * FROM group_countries
     ORDER BY total_sales DESC
    )

SELECT
    country,
    number_of_customers,
    total_sales,
    avg_order_value,
    avg_sales
FROM
    (
     SELECT
        m.*,
        CASE
            WHEN m.country == 'other' THEN 1
            ELSE 0
        END AS sort
     FROM merged_data m
     ORDER BY m.total_sales DESC
    )
ORDER BY sort

Done.


country,number_of_customers,total_sales,avg_order_value,avg_sales
USA,13,1040.4899999999998,7.942671755725189,80.0376923076923
Canada,8,535.5900000000001,7.047236842105265,66.94875000000002
Brazil,5,427.68000000000006,7.0111475409836075,85.53600000000002
France,5,389.0699999999999,7.781399999999998,77.81399999999998
Germany,4,334.62,8.161463414634147,83.655
Czech Republic,2,273.24000000000007,9.108000000000002,136.62000000000003
United Kingdom,3,245.52,8.768571428571429,81.84
Portugal,2,185.13000000000002,6.383793103448276,92.56500000000001
India,2,183.14999999999998,8.72142857142857,91.57499999999999
other,15,1094.9399999999998,7.445071062271063,72.996



Based on the data, there may be opportunity in the following countries:

- Czech Republic
- United Kingdom
- India

It's worth remembering this because the amount of data from each of these countries is relatively low. As such, we should be cautious about spending too much money on new marketing campaigns because the sample size isn't large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

In [12]:
%%sql
SELECT
    Case
        WHEN (
              SELECT t.track_id
              FROM invoice_line il
              LEFT JOIN track t ON t.track_id =  il.track_id
              WHERE il.invoice_id = i.invoice_id
            
              EXCEPT
            
              SELECT track_id
              FROM track
              WHERE album_id = (
                                SELECT MIN(album_id)
                                FROM invoice_line il
                                LEFT JOIN track t ON il.track_id = t.track_id
                                WHERE il.invoice_id = i.invoice_id
                               )
             ) IS NULL
         AND
             (
              SELECT track_id
              FROM track
              WHERE album_id = (
                                SELECT MIN(album_id)
                                FROM invoice_line il
                                LEFT JOIN track t ON il.track_id = t.track_id
                                WHERE il.invoice_id = i.invoice_id
                               )
              EXCEPT
                 
              SELECT t.track_id
              FROM invoice_line il
              LEFT JOIN track t ON t.track_id =  il.track_id
              WHERE il.invoice_id = i.invoice_id
             ) IS NULL
        THEN 'ALBUM'
        ELSE 'TRACK'
    END AS 'Purchase Type',
    COUNT(*) number_of_invoices,
    COUNT(*) / CAST((SELECT COUNT(*) FROM invoice) AS FLOAT) percentage 
FROM invoice i
GROUP BY 1

Done.


Purchase Type,number_of_invoices,percentage
ALBUM,114,0.18566775244299674
TRACK,500,0.8143322475570033


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.